In [1]:
import os
import pandas as pd

In [2]:
RANDOM_SEED = 43

Reading Data

In [3]:
df = pd.read_csv("dataset/nlbse-2023-issue-report.csv")
df.shape
df = df[df['labels']=='bug']
print(df.shape)
df_unique = df.drop_duplicates(subset='id', keep='first')
print(df_unique.shape)
df = df_unique
df["id"] = df["id"].astype(int)
df_selected = df[['id','body']]

concatenated_df = pd.read_csv("dataset/Manual-labelled-data-25K.csv")
concatenated_df_selected = concatenated_df[['Issue_id','Candidate_String', 'is_secret_human_label']]

# Rename columns for consistency
df_selected = df_selected.rename(columns={'body': 'text'})
concatenated_df_selected = concatenated_df_selected.rename(columns={'Candidate_String': 'candidate_string', 'is_secret_human_label': 'label'})

# Merge the two DataFrames on a common key, for example, 'id'
# Adjust the merge key as needed based on your data
merged_df = concatenated_df_selected.merge(df_selected, left_on='Issue_id', right_on='id')
print(merged_df.shape)
columns_to_remove = ['id']
merged_df.drop(columns=columns_to_remove, inplace=True)
print(merged_df.columns)



C:\Users\ahmed\AppData\Local\Temp\ipykernel_20320\2050619644.py:1: DtypeWarning: Columns (0,1) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("dataset/nlbse-2023-issue-report.csv")


(745732, 6)
(726503, 6)
(25451, 5)
Index(['Issue_id', 'candidate_string', 'label', 'text'], dtype='object')


Preprocessing Input

In [13]:
#print(merged_df.size)
#print(merged_df.shape[0])
#print(type(merged_df))
#print(merged_df.head())
preprocessed_df=None
unprocessed_df = None
count=0
inverse_count=0
for i in range(merged_df.shape[0]):
  main_string=merged_df['text'][i]
  substring=merged_df['candidate_string'][i]
  if main_string.find(substring)== -1 :
    count+=1
    if unprocessed_df is None:
      unprocessed_df = pd.DataFrame(merged_df.iloc[i, :]).transpose()
    else:
      unprocessed_df = pd.concat([unprocessed_df, pd.DataFrame(merged_df.iloc[i, :]).transpose()], ignore_index=True)
  else:
    inverse_count+=1
    # Append the row to preprocessed_df
    if preprocessed_df is None:
      preprocessed_df = pd.DataFrame(merged_df.iloc[i, :]).transpose()
    else:
      preprocessed_df = pd.concat([preprocessed_df, pd.DataFrame(merged_df.iloc[i, :]).transpose()], ignore_index=True)

print("Mismatch: "+str(count))
print(unprocessed_df.shape)
#print(unprocessed_df.head())

print("Ok: "+str(inverse_count))
print(preprocessed_df.shape)
#print(preprocessed_df.head())


Mismatch: 1807
(1807, 4)
Ok: 23644
(23644, 4)


In [14]:
import re

for j in unprocessed_df.index:
        # if df["id"][j] != "1165939311":
        #         continue
        input_string =    str(df[df["id"]==unprocessed_df["Issue_id"][j]]["body"])    
        input_string = re.sub(r'[\'"\│]', '', input_string)
        dir_list_clean = re.sub(r'drwx[-\s]*\d+\s+\w+\s+\w+\s+\d+\s+\w+\s+\d+\s+[0-9a-fA-F-]+.*','',input_string)
        shell_code_free_text = re.sub(r'```shell([^`]+)```','',dir_list_clean,flags=re.IGNORECASE)
        shell_code_free_text = re.sub(r'```Shell\s*"([^"]*)"\s*```','',shell_code_free_text,flags=re.IGNORECASE)
        # saved_game_free_text = re.sub(r'```([^`]+)```','',shell_code_free_text) #etay jhamela hobe
        saved_game_free_text = re.sub(r'<details><summary>Saved game</summary>\n\n```(.*?)```', '', shell_code_free_text)
        remove_packages = re.sub(r'(\w+\.)+\w+','',saved_game_free_text)
        java_exp_free_text = re.sub(r'at\s[\w.$]+\.([\w]+)\(([^:]+:\d+)\)','',remove_packages)
        # url_free_text= re.sub(https?://[^\s#]+#[A-Za-z0-9\-]+,'', java_exp_free_text, flags=re.IGNORECASE)
        url_with_fragment_text= re.sub(r'https?://[^\s#]+#[A-Za-z0-9\-\=\+]+','', java_exp_free_text, flags=re.IGNORECASE)
        url_free_text= re.sub(r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\\(\\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', '',url_with_fragment_text)
        commit_free_text= re.sub(r'commit[ ]?(?:id)?[ ]?[:]?[ ]?([0-9a-f]{40})\b', '', url_free_text, flags=re.IGNORECASE)
        file_path_free_text = re.sub(r"/[\w/. :-]+",'',commit_free_text)
        file_path_free_text = re.sub( r'(/[^/\s]+)+','',file_path_free_text)
        sha256_free_text = re.sub(r'sha256\s*[:]?[=]?\s*[a-fA-F0-9]{64}','',file_path_free_text)
        sha1_free_text = re.sub(r'git-tree-sha1\s*=\s*[a-fA-F0-9]+','',sha256_free_text)
        build_id_free_text = re.sub(r'build-id\s*[:]?[=]?\s*([a-fA-F0-9]+)','',sha1_free_text)
        guids_free_text = re.sub(r'GUIDs:\s+([0-9a-fA-F-]+\s+[0-9a-fA-F-]+\s+[0-9a-fA-F-]+)','',build_id_free_text)
        uuids_free_text = re.sub(r'([0-9a-fA-F-]+\s*,\s*[0-9a-fA-F-]+\s*,\s*[0-9a-fA-F-]+)','',guids_free_text)
        event_id_free_text = re.sub(r'<([^>]+)>','',uuids_free_text)
        UUID_free_text = re.sub(r'(?:UUID|GUID|version|id)[\\=:"\'\s]*\b[a-fA-F0-9]{8}-[a-fA-F0-9]{4}-[a-fA-F0-9]{4}-[a-fA-F0-9]{4}-[a-fA-F0-9]{12}\b'
,'',event_id_free_text,flags=re.IGNORECASE) ##without the prefix so many false positives can be omitted
        hex_free_text = re.sub(r'(?:data|address|id)[\\=:"\'\s]*\b0x[0-9a-fA-F]+\b','',UUID_free_text,flags=re.IGNORECASE) ## deleting hex ids directly can cause issues
        ss_free_text = re.sub(r'Screenshot_(\d{4}[_-]\d{2}[_-]\d{2}[_-]\d{2}[_-]\d{2}[_-]\d{2}[_-]\d{2}[_-]\w+)','',hex_free_text,flags=re.IGNORECASE)
        cleaned_text = ss_free_text

        #unprocessed_df.loc[j, 'Issue_id']=cleaned_text
        unprocessed_df.loc[j,'Issue_id'] = str(df[df["id"]==unprocessed_df["Issue_id"][j]]["body"])

In [17]:
count=0
inverse_count=0

for i in range(unprocessed_df.shape[0]):
  #print(i)
  main_string=unprocessed_df['text'][i]
  substring=unprocessed_df['candidate_string'][i]
  #print(main_string.find(substring))
  if main_string.find(substring)!=-1:
    count+=1
  else:
    inverse_count+=1
print("Mismatch: "+str(inverse_count))
print("Ok: "+str(count))
print(unprocessed_df.head())

Mismatch: 1807
Ok: 0
                                            Issue_id  \
0  135024    ### Did you clear cache before openi...   
1  579753    ### Update disclaimer\n\n- [x] Yes, ...   
2  1038525    ### Describe the bug\r\n\r\nDeployi...   
3  1024594    **[Dario Scoppelletti](https://jira...   
4  720705    用户分级scopes字段不管用\r\n`package org.spri...   

                                    candidate_string label  \
0  showTimerProgress:false,smoothCaret:true,quick...     0   
1                           c__DisplayClass24_0.b__1     0   
2                                   ;pw=();print((pw     0   
3                         xAC\xED\x00\x05sr\x00\\x95     0   
4                             serviceToken).toString     0   

                                                text  
0  ### Did you clear cache before opening an issu...  
1  ### Update disclaimer\n\n- [x] Yes, I have che...  
2  ### Describe the bug\r\n\r\nDeploying with the...  
3  **[Dario Scoppelletti](https://jira.spring.io/

In [61]:

# processed_df = pd.concat([unprocessed_df,preprocessed_df])

Setting Context Window

In [111]:
def create_context_window(text, target_string, window_size=200):

    target_index = text.find(target_string)
    #print(target_index)

    if target_index != -1:
        start_index = max(0, target_index - window_size)
        end_index = min(len(text), target_index + len(target_string) + window_size)
        context_window = text[start_index:end_index]
        return context_window

    return None

# Apply the create_context_window function to each row in the DataFrame
preprocessed_df['modified_text'] = preprocessed_df.apply(lambda row: create_context_window(row['text'], row['candidate_string']), axis=1)
print(preprocessed_df.shape)
print(preprocessed_df['modified_text'].head())

(25451, 5)
0    pute-1.amazonaws.com> SSH: EXEC ssh -vvv -o Co...
1    mplete the following information):**\r\n - Dev...
2                                                 None
3    ### What happened?\n\ngws-ingress services kee...
4    ain() {\r\n  runApp(const MyApp());\r\n}\r\n\r...
Name: modified_text, dtype: object


In [63]:
#pip install transformers

In [89]:
import torch
from transformers import RobertaTokenizer, RobertaForSequenceClassification, AdamW
from torch.utils.data import DataLoader, TensorDataset, Dataset
from sklearn.model_selection import train_test_split
import numpy as np

Setting Seeds

In [90]:
torch.manual_seed(RANDOM_SEED)
torch.cuda.manual_seed_all(RANDOM_SEED)
np.random.seed(RANDOM_SEED)

## Processing Data

In [91]:
# Load and prepare your dataset
# You will have two lists: X_text (text bodies) and X_candidate (candidate strings)

# Encode text bodies and candidate strings separately
X_issue_ids = preprocessed_df['Issue_id'].tolist()
X_text = preprocessed_df['text'].tolist()  # Convert the 'text' column to a list of strings
X_candidate = preprocessed_df['candidate_string'].tolist()  # Convert the 'candidate_string' column to a list of strings
Y_labels = preprocessed_df['label'].tolist()

In [67]:
# X_text = X_text[0:100]
# X_candidate = X_candidate[0:100]
# Y_labels = Y_labels[0:100]

In [92]:
split_ratio = 0.2
X_text_train, X_text_test, X_candidate_train, X_candidate_test, Y_labels_train, Y_labels_test,X_issue_ids_train,X_issue_ids_test = train_test_split(
        X_text, X_candidate, Y_labels,X_issue_ids, test_size=split_ratio, random_state=RANDOM_SEED
)

In [69]:
#type(Y_labels_test), len(Y_labels_test)

In [70]:
#print(len(X_text_train), len(X_text_test))

In [71]:
#!nvidia-smi

In [93]:
# Tokenize text bodies and candidate strings separately
def encode_texts(texts):
    encodings = tokenizer(texts, padding=True, truncation=True, return_tensors='pt', max_length=512)
    return encodings

tokenizer = RobertaTokenizer.from_pretrained("roberta-base")

In [94]:
text_body_encodings = encode_texts(X_text_train)
candidate_encodings = encode_texts(X_candidate_train)

In [ ]:
# Define a custom dataset class
class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, text_encodings, candidate_encodings, labels):
        self.text_encodings = text_encodings
        self.candidate_encodings = candidate_encodings
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx): #it works fine for training
        text_input_ids = self.text_encodings['input_ids'][idx]
        text_attention_mask = self.text_encodings['attention_mask'][idx]
        candidate_input_ids = self.candidate_encodings['input_ids'][idx]
        candidate_attention_mask = self.candidate_encodings['attention_mask'][idx]
        label = torch.tensor(self.labels[idx])

        return text_input_ids, text_attention_mask, candidate_input_ids, candidate_attention_mask, label

# train_dataset = CustomDataset(text_body_encodings, candidate_encodings, Y_train)
# test_dataset = CustomDataset(X_test_text, X_test_candidate, Y_test)

In [ ]:
np.unique(np.asarray(Y_labels_train), return_counts=True)

In [ ]:
print(type(Y_labels_train), len(Y_labels_train))
Y = np.array(Y_labels_train)
Y_ =Y.astype(int)

In [ ]:
train_dataset = CustomDataset(text_body_encodings, candidate_encodings, Y_)

In [ ]:
train_loader = DataLoader(train_dataset, batch_size=8, shuffle=False)
print(len(train_loader))

In [ ]:
type(np.array(Y_labels_train))


In [ ]:

torch.cuda.is_available()
torch.cuda.get_device_name(0)

## Fine-tune RoBERTa

In [ ]:
model = RobertaForSequenceClassification.from_pretrained("roberta-base", num_labels=2)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

Training Model

In [ ]:
# Fine-tune RoBERTa


#optimizer = torch.optim.AdamW(model.parameters(), lr=1e-5)



# learning_rate = 1e-3
# momentum = 0.9  # If using SGD with momentum
# optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, momentum=momentum)


# learning_rate = 1e-5
# optimizer = torch.optim.Adagrad(model.parameters(), lr=learning_rate)



learning_rate = 1e-5
optimizer = torch.optim.RMSprop(model.parameters(), lr=learning_rate)

num_epochs = 10

# Training loop
model.train()
for epoch in range(num_epochs):
    c = 1
    # for batch in train_loader:
    #     print("batch: ",c)
    #     text_input_ids, text_attention_mask, candidate_input_ids, candidate_attention_mask, labels = batch
    #     text_input_ids, text_attention_mask, candidate_input_ids, candidate_attention_mask, labels = text_input_ids.to(device), text_attention_mask.to(device), candidate_input_ids.to(device), candidate_attention_mask.to(device), labels.to(device)

    #     outputs = model(input_ids=text_input_ids, attention_mask=text_attention_mask, labels=labels)
    #     loss = outputs.loss
    #     loss.backward()
    #     optimizer.step()
    #     optimizer.zero_grad()
    for batch in train_loader:
        print("epoch: ",epoch)
        print("batch: ", c)
        text_input_ids, text_attention_mask, candidate_input_ids, candidate_attention_mask, labels = batch

        # Ensure that you access the tensors with integer indices, not string keys
        text_input_ids, text_attention_mask, candidate_input_ids, candidate_attention_mask, labels = (
            text_input_ids.to(device),
            text_attention_mask.to(device),
            candidate_input_ids.to(device),
            candidate_attention_mask.to(device),
            labels.to(device)
        )
        with torch.no_grad():
            for module in model.modules():
                if isinstance(module, torch.nn.Dropout):
                    module.p = 0  # Set dropout probability to 0

        # The rest of your training code
        outputs = model(input_ids=text_input_ids.type(torch.LongTensor).cuda(), attention_mask=text_attention_mask.type(torch.LongTensor).cuda(), labels=labels.type(torch.LongTensor).cuda())
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        c = c+1

# Evaluate the model
model.eval()
# with torch.no_grad():
#     for batch in test_loader:
#         text_input_ids, text_attention_mask, candidate_input_ids, candidate_attention_mask, labels = batch
#         text_input_ids, text_attention_mask, candidate_input_ids, candidate_attention_mask, labels = text_input_ids.to(device), text_attention_mask.to(device), candidate_input_ids.to(device), candidate_attention_mask.to(device), labels.to(device)

#         outputs = model(input_ids=text_input_ids, attention_mask=text_attention_mask)
#         predicted_labels = torch.argmax(outputs.logits, dim=1)



Saving and loading model

In [ ]:
# Save the model to a file
model_path = "models/local_roberta_model_10epoch_lr1e-5_torchrmsprop_cntxt200_data25k.pth"
torch.save(model.state_dict(), model_path)


In [ ]:
# Load the model from the file
model = RobertaForSequenceClassification.from_pretrained("roberta-base", num_labels=2)  # Replace with your model configuration
model.load_state_dict(torch.load(model_path))
model.eval()  # Set the model to evaluation mode for inference


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

## Test Dataset here

In [ ]:
text_body_encodings_test = encode_texts(X_text_test)
candidate_encodings_test = encode_texts(X_candidate_test)

In [ ]:
print(type(Y_labels_test), len(Y_labels_test))
Y_test = np.array(Y_labels_test)
print(type(Y_test), len(Y_test))
print(Y_test[:4085])
#for i in range(len(Y_test)):
#  if Y_test[i] ==  ' AWS4-HMAC-SHA256 Credential':
#    Y_test[i]='1'
Y_test_ =Y_test.astype(int)


In [ ]:

test_dataset = CustomDataset(text_body_encodings_test, candidate_encodings_test, Y_test_)

In [ ]:
test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False)

Testing model

In [ ]:
model.to(device)
model.eval()  # Set the model in evaluation mode
c=0
predicted_labels_list = []
with torch.no_grad():

    for batch in test_loader:
        print("Batch %d"%c)
        c+=1

        text_input_ids, text_attention_mask, candidate_input_ids, candidate_attention_mask, labels = batch

        # Move tensors to the device
        text_input_ids, text_attention_mask, candidate_input_ids, candidate_attention_mask, labels = (
            text_input_ids.to(device),
            text_attention_mask.to(device),
            candidate_input_ids.to(device),
            candidate_attention_mask.to(device),
            labels.to(device)
        )

        # Perform inference

        outputs = model(input_ids=text_input_ids.type(torch.LongTensor).cuda(), attention_mask=text_attention_mask.type(torch.LongTensor).cuda())
        predicted_labels = torch.argmax(outputs.logits, dim=1)

        # print(f"predicted_labels: {predicted_labels}")
        predicted_labels_list.append(predicted_labels[0])

In [ ]:
predicted_labels_list_output = [f.cpu().numpy().tolist() for f in predicted_labels_list]

In [ ]:
print(len(X_candidate_test))
print(len(Y_labels_test))
print(len(predicted_labels_list_output))
print(len(X_issue_ids_test))
print(len(X_text_test))
test_text=[]
test_id=[]
test_can=[]
test_label=[]
test_pred=[]
for i in range(len(Y_labels_test)):
  if Y_labels_test[i] == '1':
    test_text.append((X_text_test[i].replace("\n", "")).replace("\r", ""))
    test_id.append(X_issue_ids_test[i])
    test_can.append(X_candidate_test[i])
    test_label.append(Y_labels_test[i])
    test_pred.append(predicted_labels_list_output[i])


dict = {'Issue Id':test_id ,'Body':test_text,'Candidate String':test_can,'Actual Label': test_label, 'Predicted Label': test_pred}
df = pd.DataFrame(dict)
df.to_csv('positive_data.csv', sep=',', index=False)


Confusion Matrix

In [ ]:
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
cm = confusion_matrix(Y_test_, predicted_labels_list_output, labels=[0,1])
disp = ConfusionMatrixDisplay(confusion_matrix=cm,display_labels=[0,1])
disp.plot()
plt.title("AdamW,contextwindow,LR=1e-5")
plt.show()

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

In [ ]:
def get_metrics(y_true, y_pred):
    accuracy = accuracy_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred)
    recall = recall_score(y_true, y_pred)
    return {
        "accuracy": accuracy,
        "precision": precision,
        "f1": f1,
        "recall": recall
    }

In [ ]:
metrics = get_metrics(Y_test_, predicted_labels_list_output)

Normal Metrics

In [ ]:
metrics

In [ ]:
def get_minority_metrics(y_true, y_pred, minority_class_label):
    # Convert lists to NumPy arrays
    y_true = np.array(y_true)
    y_pred = np.array(y_pred)

    # Calculate the metrics for the minority class
    minority_indices = np.where(y_true == minority_class_label)
    y_true_minority = y_true[minority_indices]
    y_pred_minority = y_pred[minority_indices]

    # Calculate precision, recall, and F1-score for the minority class
    minority_precision = precision_score(y_true_minority, y_pred_minority)
    minority_recall = recall_score(y_true_minority, y_pred_minority)
    minority_f1 = f1_score(y_true_minority, y_pred_minority)

    return {
        'Minority Precision': minority_precision,
        'Minority Recall': minority_recall,
        'Minority F1-Score': minority_f1
    }

In [ ]:
minority_metrics = get_minority_metrics(Y_test_, predicted_labels_list_output, 1)

Minority Metrics

In [ ]:
minority_metrics